# Maximizing Revenue for Taxi Cab Drivers through Payment Type Analysis

## Problem statement
In the fast-paced taxi booking sector, making the most of the revenue is essential for long-term success and driver happiness. Our goal is to use data driven insights to maximize the revenue streams for taxi-drivers in order to meet this need Our research aims to determine whether payment methdos have an impact on fare pricing by focusing on the relationship between payment type and fare amount 

### Objective
This project's main goal is to run an A/B test to examine the relationship between the total fare ansd the method of payment. We use Python hypothesis testing and descriptive statistics to extract useful information that can help taxi drivers generate more cash. In particular, we want to find out if there's a big difference in the fares for those who pay with credit cards versus those who pay with cash

### Research Question
Is there a relationship between total fare amount and payment type and can we judge customers towards payment methods that generate higher revenue for drivers, without negatively impacting customer experience

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import warnings

warnings.filterwarnings('ignore')

#### Loading the Data

In [2]:
#!pip install opendatasets
#import opendatasets as od
#link = 'https://data.cityofnewyork.us/api/views/kxp8-n2sj/rows.csv?accessType=DOWNLOAD'
#od.download(link)
df = pd.read_csv('2023_Yellow_Taxi_Trip_Data.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,01/01/2023 12:03:48 AM,01/01/2023 12:13:25 AM,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38310226 entries, 0 to 38310225
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
 18  airport_fee            float64
dtypes: float64(12), int64(4), object(3)
memory usage: 5.4+ GB


In [4]:
df = df.sample(n=6000000, random_state=42)

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])



df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()
df['duration'] = df['duration'] / 60
df.head()

df = df[['passenger_count', 'payment_type', 'fare_amount', 'trip_distance', 'duration']]

df.head()

KeyboardInterrupt: 

In [ ]:
df.isna().sum()

In [ ]:
# % na in passenger_count
df.isna().sum() * 100/ len(df)

**We they are just 3.43 % so we can drop them **

In [ ]:
df.dropna(inplace = True)
print(6000000 - len(df))
df

In [ ]:
df.info()

In [ ]:
df['passenger_count'] = df[['passenger_count']].astype('int64')
len(df[df.duplicated()]) 

In [46]:
len(df) - len(df[df.duplicated()]) 
df.drop_duplicates(inplace = True)


In [ ]:
df['passenger_count'].value_counts(normalize = True)

In [ ]:
df['payment_type'].value_counts(normalize = True)

#### We are studying only for card and cash type therefore we can remove the other 2 

In [ ]:
df = df[df['payment_type'] < 3]
df

In [ ]:
df['passenger_count'].value_counts()


#### Usally it is not possible that passenger count is > 5 and 0 also doesn't make sense therefore we remove them

In [57]:

df = df[(df['passenger_count'] > 0) & (df['passenger_count'] < 6)]

#### Lets replace 1 with card and 2 with cash So that things get clear

In [ ]:
df['payment_type'].replace([1, 2], ['card', 'cash'], inplace = True)
df

In [ ]:
df.describe()

** fare amount, trip distance and duration are -ve which is not possible, So we remove them **

In [60]:
df = df[df['fare_amount'] > 0]
df = df[df['trip_distance'] > 0]
df = df[df['duration'] > 0]

In [ ]:
plt.boxplot(df['fare_amount'])

**Way too many outlier are present in the data, we have to remove them. We can use Z-score or Inter Quantile Range(IQR) Z-score can't be used because of the distribution. Therefore, We use IQR**

In [ ]:
for col in ['fare_amount', 'trip_distance', 'duration']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]

df

In [ ]:
plt.hist(df['fare_amount'])

In [64]:
df.to_csv('final_file.csv', index = False)

#### We have done Data Clearning and Transforming, We will Analysis in Next NoteBook 'analysis'